# Heuristic Methods

## Neighborhood Search

### Implementation

In [136]:
import System.Random
import Control.Monad ( replicateM)

The first thing that should be done is to write some helper-functions for generating pseudorandom numbers. To do that, I'll start by defining **seed** for random generators and a function that will generate a finite length list of values in range, `getRandomValuesInRange::Int->Int->Int->IO [Int]`. For exercise purpose only, I'll define second function that will return a finite length list of doubles, uniformly distributed on $[0,1)$, `getRandomValues::Int->[Double]`.

In [137]:
seed::Int
seed=(-958036805781772734)

In [138]:
getRandomValues :: Int -> [Double]
getRandomValues len=take len $ randoms (mkStdGen seed)::[Double]

In [139]:
getRandomValues 4

[0.3173710114340238,0.9038063995872138,0.26811089937893495,0.2091390866782773]

In [140]:
getRandomValuesInRange::Int->Int->Int->IO [Int]
getRandomValuesInRange len a b = replicateM len (getStdRandom $ randomR (a,b))

In [141]:
getRandomValuesInRange 8 0 1

[1,0,1,0,1,1,0,0]

In [158]:
newtype Item = Item { rawItem::(Int,Int) } deriving (Show,Eq)

Next step is to define a generic datatype for the possible solutions. For a simple problem like knapsack the solution can be a list of binary values.

In [159]:
newtype Solution = Solution { rawSolution::([Int],Int)} deriving (Show,Eq)

Normally, the next step should be to define a fitness function but that would make the implementation tightly coupled of the problem that solves. For example, if the implementation is needed to solve the knapsack problem, one of the parameters needed to be passed to the fitness function should be the dataset. Instead, a new type for dataset will be defined. 

For knapsack problem, dataset can be a list of tuples `(g,v)` where $g$ is the weight of the $n^{th}$ object and $v$ is the value of the object.

In [160]:
newtype Dataset = Dataset { rawDataset::(Int,[Item])} deriving (Show,Eq)

Now a generic fitness function can be defined:

In [161]:
fitness::Dataset -> Solution -> Int
fitness dataset solution = fitnessValue
    where
        datasetRaw = rawDataset dataset
        maxWeight = fst datasetRaw
        items = map rawItem (snd datasetRaw)
        individual = fst $ rawSolution solution
        pairs = zip individual items
        weight = foldl (\acc x-> if fst x==1 then acc+fst (snd x) else acc+0) 0 pairs
        value = foldl (\acc x-> if fst x==1 then acc+snd (snd x) else acc+0) 0 pairs
        fitnessValue = if weight <= maxWeight then value else 0

Ok, it's a complicated function and it's problem specific. Ok, maybe not so complicated but maybe not good written. Let's take it step by step:

1. It accepts 2 parameters of types Dataset and Solution, respectively
2. `datasetRaw = rawDataset dataset` -> get the tuple (G,[(g,v)]), where G is Maximum Weight, g is item's weight and v is item's value.
3. `maxWeight = fst datasetRaw` -> get the maximum weight from the tuple
4. `items = map rawItem (snd datasetRaw)` -> get the list [(g,v)]
5. `individual = fst $ rawSolution solution` -> extract the individual configuration from solution
6. `pairs = zip individual items` -> create pairs of type [(is_item,(g,v))]
7. `weight = foldl (\acc x-> if fst x==1 then acc+fst (snd x) else acc+0) 0 pairs` -> extract total weight for solution
8. `value = foldl (\acc x-> if fst x==1 then acc+snd (snd x) else acc+0) 0 pairs` -> extract total value for solution
9. `fitnessValue = if weight <= maxWeight then value else 0`

Now the implementation of the function that will calculate the neighborhood can be done. The function will receive as parameters a fitness function, the dataset, current solution and will return a new solution. In other words, the function's type will be `getNewNeighbor::(Dataset->Solution->Double)->Dataset->Solution->Solution`

 ```python
 solutions=list()
    for i in range(0,len(S)):
        new_solution=list(S)
        if S[i] == 0:
            new_solution[i] = 1
        else:
            new_solution[i] = 0
        value = cost_function(new_solution,max_weight,values,weights)
        solutions.append((new_solution,value))
    best_solution=max(solutions,key=lambda item:item[1])
    return best_solution[0],best_solution[1]
```

In [162]:
replaceNth::Int -> Int->[Int]->[Int]
replaceNth _ _ [] = []
replaceNth index newVal (x:xs)
    | index==0 = newVal:xs
    | otherwise = x:replaceNth (index-1) newVal xs

In [191]:
maximum'::[([Int],Int)]->([Int],Int)
maximum' [] = error "empty list"
maximum' (x:xs) = maxTail x xs
    where maxTail currentMax [] = currentMax
          maxTail (m,n) (p:ps)
              | n<(snd p) = maxTail p ps
              | otherwise = maxTail (m,n) ps

In [195]:
calculateNeighborhood::(Dataset->Solution->Int)->Dataset->Solution->Int->[([Int],Int)]->[([Int],Int)]
calculateNeighborhood _ _ _ 0 neighbors = neighbors
calculateNeighborhood f dataset solution len neighbors = calculateNeighborhood f dataset solution (len-1) (neighbor:neighbors)
    where
        raw = fst $ rawSolution solution
        newNeighbor = if raw !! (len-1) == 1 then replaceNth (len-1) 0 raw else replaceNth (len-1) 1 raw
        fitnessValue = f dataset (Solution (newNeighbor,0))
        neighbor = (newNeighbor,fitnessValue)

In [196]:
getNewNeighbor::(Dataset->Solution->Int)->Dataset->Solution->Solution
getNewNeighbor f dataset solution = result 
    where
        rawList = fst $ rawSolution solution
        neighbors = calculateNeighborhood f dataset solution (length rawList) []
        result = Solution (maximum' neighbors)
        

In [197]:
a=Solution ([0,1,1,1,1],0)

In [198]:
b=Dataset (10,[Item (1,1), Item (2,2),Item (3,3),Item (4,4),Item (5,5)])

In [199]:
getNewNeighbor fitness b a

Solution {rawSolution = ([0,1,1,0,1],10)}

In [200]:
calculateNeighborhood fitness b a 5 []

[([1,1,1,1,1],0),([0,0,1,1,1],0),([0,1,0,1,1],0),([0,1,1,0,1],10),([0,1,1,1,0],9)]

In [187]:
a=[([1,1,1,1,1],0),([0,1,1,0,1],20),([0,1,0,1,1],0),([0,1,1,0,1],10),([0,1,1,1,0],9)]

In [188]:
a

[([1,1,1,1,1],0),([0,1,1,0,1],20),([0,1,0,1,1],0),([0,1,1,0,1],10),([0,1,1,1,0],9)]

In [189]:
maximum'::[([Int],Int)]->([Int],Int)
maximum' [] = error "empty list"
maximum' (x:xs) = maxTail x xs
    where maxTail currentMax [] = currentMax
          maxTail (m,n) (p:ps)
              | n<(snd p) = maxTail p ps
              | otherwise = maxTail (m,n) ps

In [190]:
maximum' a

([0,1,1,0,1],20)